In [ ]:
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.layers import Conv2D, GlobalAveragePooling2D, \
                                    Dense, BatchNormalization, Activation, \
                                    DepthwiseConv2D

In [ ]:
input_shape = (224, 224, 3)
classes_num = 1000

In [ ]:
def depth_wise_separable_conv(x, filter, stride, name=None):
  x = DepthwiseConv2D(3, stride, 'same', use_bias=False)(x)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)
  x = Conv2D(filter, 1, 1, 'same', use_bias=False)(x)
  x = BatchNormalization()(x)
  return Activation('relu', name=name)(x)

In [ ]:
input = keras.Input(shape=input_shape)

x = Conv2D(32, 3, 2, 'same', use_bias=False)(input)
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = depth_wise_separable_conv(x, 64, 1, 'depth_wise_sep_conv_1')
x = depth_wise_separable_conv(x, 128, 2, 'depth_wise_sep_conv_2')
x = depth_wise_separable_conv(x, 128, 1, 'depth_wise_sep_conv_3')
x = depth_wise_separable_conv(x, 256, 2, 'depth_wise_sep_conv_4')
x = depth_wise_separable_conv(x, 256, 1, 'depth_wise_sep_conv_5')
x = depth_wise_separable_conv(x, 512, 2, 'depth_wise_sep_conv_6')

for idx in range(5):
  x = depth_wise_separable_conv(x, 512, 1,
      'depth_wise_sep_conv_{}'.format(idx+7))

x = depth_wise_separable_conv(x, 1024, 2, 'depth_wise_sep_conv_12')
x = depth_wise_separable_conv(x, 1024, 1, 'depth_wise_sep_conv_13')

x = GlobalAveragePooling2D()(x)
output = Dense(classes_num, activation='softmax')(x)

In [ ]:
model = keras.Model(
    inputs=input,
    outputs=output
)

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 112, 112, 32)      864       
_________________________________________________________________
batch_normalization (BatchNo (None, 112, 112, 32)      128       
_________________________________________________________________
activation (Activation)      (None, 112, 112, 32)      0         
_________________________________________________________________
depthwise_conv2d (DepthwiseC (None, 112, 112, 32)      288       
_________________________________________________________________
batch_normalization_1 (Batch (None, 112, 112, 32)      128       
_________________________________________________________________
activation_1 (Activation)    (None, 112, 112, 32)      0     